<p> <b> Tarea 1: Machine Learning </b> </p>
<p> Juan Sebastian Beleño Torregoza [codigo] Jose Alejandro Cubillos Muñoz 201313719 </p>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import linear_model
#import os
from sklearn.model_selection import train_test_split

<p><b> Punto 1:  </b></p>
<p><b> Se hará una regresión por cada marca de carro registrada en la lista "marcas" </b><p>

In [2]:
#marcas = os.popen('ls ./carDataset/').read().split('.csv\n')[:-1]
"""
Lista de marcas con las que se va a trabajar. Si se tienen otros datos con otras marcas cuyo archivo .csv está en el mismo directorio 
de trabajo que el presente archivo, descomentar la línea anterior. Esta solo funciona en Linux. En Windows especifique la marca en la siguiente línea
"""
marcas = ['audi', 'bmw', 'ford', 'hyundi', 'merc', 'skoda', 'toyota', 'vauxhall', 'vw']
print(marcas)

['audi', 'bmw', 'ford', 'hyundi', 'merc', 'skoda', 'toyota', 'vauxhall', 'vw']


<p> En la siguiente celda se declaran las funciones que transformarán los datos dados en el .csv, aplicándoles el método "One Hot" a los datos categóricos. La función encargada de aplicar el "One Hot" se llama "cocinar_categoricos". Los datos también entran en un proceso de normalización. La función encargada de normalizar se llama "normalizar". Una vez se definen las funciones se aplica la función de "cocinar_categoricos" a todos los datos. Luego se dividen los datos aleatoriamente en entrenamiento y test (80% para entrenamiento, 20% para test). FInalmente se normalizan tanto los datos de test como de entrenamiento con la media y la desviación de los datos de entrenamiento. </p>

In [3]:
def cocinar_categoricos(dataSet, lista_categoricos):
    """
    Esta función toma como entrada un data frame y guarda todos los posibles valores de las columnas especificadas. A estas columnas las llamaremos columnas antiguas
    Luego crea nuevas columnas con el nombre de los posibles valores y le asigna a cada dato de las nuevas columnas un 1 si  el dato en la columna antigua tiene el valor del nombre de
    la columna nueva, 0 de lo contrario. Finalmente se borran las columnas antiguas
    @input: dataSet <pd.DataFrame>: Datos que se van a modificar por el método One Hot
    @input: lista_catogóricos <list>: lista de "features" o columnas a las que se va a aplicar el métodode One Hot
    """
    for feature in lista_categoricos:
        if feature in dataSet.keys():
            posibles_valores = set(dataSet[feature])
            for valor in posibles_valores:
                dataSet[valor] = list(map(lambda x: 1 if x==valor else 0, dataSet[feature]))
            #fin for
        #fin if
        del dataSet[feature]
    #fin for 
#fin funcion

def normalizar(dataSet, features, parametros = []):
    """
    Esta función resta la media y luego divide entre la desviación estandar de cada dato en cada columna. Si se le especifica la media y la desviación en los parámetros 
    Usa esa media y desviación dada, de lo contrario, calcula la media y la desviación de cada columna especificada en "features"
    @input: dataSet <pd.DataFrame>: Tabla de datos a la que se le va a dar normalización a la columans dadas
    @input: features <list>: lista de "features" o columnas que se van a normalizar
    @input: (opcional) parametros <dict>: En caso de que se quiera normalizar con una media y una desviación dada se especifican en un diccionario así: {'mileage':{'media': 2.9, 'desviacion': 7}}
    """
    if len(parametros) == 0:
        for feature in features:
            media = np.mean(dataSet[feature])
            desviacion = np.std(dataSet[feature])
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
    #fin if 
    else:
        for feature in features:
            media = parametros[feature]['media']
            desviacion = parametros[feature]['desviacion']
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
#fin función

"""
En esta parte se definen los features categóricos a los cuales se les va a aplicar el "One Hot" (features_categoricos) y los faetures, los cuales se van normalizar (features_normalizar)
Note que los categóricos no se normalizaron. Esto se hizo debido a la gran cantidad de datos que hay de cada marca. Al normalizar un dato categórico el cero se convierte en un número 
negativo muy pequeño y el 1 en un número positivo también muy pequeño, lo cual afecta tanto al modelo de scikit learn como al modelo propio implementado con descenso de gradiente estocástico.
Por esta razón se dejaron los datos categóricos como 0 o 1, lo cual arroja mejores resultados. 

Una vez se entra en el bucle de "for marca in marcas" se aplica el "One hot" a todos los datos, se dividen los datos de entrenamiento y de test, se guarda la media y la desviación de
todas las columnas de los datos de entrenamiento y finalmente se normaliza los datos de entrenamiento y los de test con las medias y desviaciones guardadas
"""

DatosTotales = dict()
DatosEntrenamiento = dict()
DatosTest = dict()
DatosEstadisticos_Entrenamiento = dict()
features_categoricos = ['model', 'transmission', 'fuelType']
features_normalizar = ['year', 'mileage', 'tax', 'mpg', 'engineSize', 'price']
for marca in marcas:
    DatosTotales[marca] = pd.read_csv('carDataset/' + marca + '.csv')
    #Aquí se aplica el "One Hot" a todos los datos 
    cocinar_categoricos(DatosTotales[marca], features_categoricos)
    DatosEntrenamiento[marca], DatosTest[marca] = train_test_split(DatosTotales[marca], test_size = 0.2)
    DatosEstadisticos_Entrenamiento[marca] = dict(map(lambda x: (x, {'media': np.mean(DatosTotales[marca][x]), 'desviacion': np.std(DatosEntrenamiento[marca][x])}), 
        DatosEntrenamiento[marca].keys()))
    #features_normalizar = DatosEntrenamiento[marca].keys()
    normalizar(DatosEntrenamiento[marca], features_normalizar)
    normalizar(DatosTest[marca], features_normalizar, parametros=DatosEstadisticos_Entrenamiento[marca])
#fin for 
#print(DatosEntrenamiento['skoda'])

<b> Entrenamiento del modelo usando scikit-learn: </b>

<p><b> En las siguientes dos celdas se define cuales columnas son los datos de entrada (X) y cual columna es el target (el precio, y) tanto para los datos de entrenamiento como para los 
de test. Luego se entrena el modelo con regresión lineal usando scikit-learn en la línea "obj_modelo_reg.fit(X_entrenamiento[marca], y_entrenamiento[marca])". Finalmente se guardan los pesos "w" y el intercepto, determinados con los datos de entrenamiento, en un diccionario llamado "resultados_sklearn". También se guarda el MSE, calculados con los datos del target de test y los "w" e interpecto aplicados al X de test, en el diccionario "MSE_p1", que significa MSE punto 1.</b></p>

In [4]:
feature_y = 'price'

In [5]:
w_sklearn = dict()
resultados_sklearn = dict()
X_entrenamiento = dict()
y_entrenamiento = dict()
X_test = dict()
y_test = dict()
y_gorro_test = dict()
MSE_p1 = dict()
for marca in marcas:
    lista_features_x = list(DatosEntrenamiento[marca].keys())
    lista_features_x.remove(feature_y)
    X_entrenamiento[marca] = DatosEntrenamiento[marca][lista_features_x]
    y_entrenamiento[marca] = DatosEntrenamiento[marca][feature_y]
    X_test[marca] = DatosTest[marca][lista_features_x]
    y_test[marca] = DatosTest[marca][feature_y]
    obj_modelo_reg = linear_model.LinearRegression()
    obj_modelo_reg.fit(X_entrenamiento[marca], y_entrenamiento[marca])
    w_sklearn[marca] = [obj_modelo_reg.intercept_, *obj_modelo_reg.coef_]
    y_gorro_test[marca] = X_test[marca].dot(w_sklearn[marca][1:]) + w_sklearn[marca][0]
    MSE_p1[marca] = 1/(len(y_test[marca]))*((y_test[marca] - y_gorro_test[marca]).dot(y_test[marca] - y_gorro_test[marca]))
    resultados_sklearn[marca] = dict(dict(map(lambda x,y: (x,y), ['intercepto'] + lista_features_x, w_sklearn[marca])))
#fin for 

<p> En esta celda se reportan los "w" para cada "feature" para cada marca. También se reporta el MSE de cada marca con los "w" determinados con scikit-learn </p>

In [6]:
for marca in marcas:
    print(marca, ": ", resultados_sklearn[marca])
#fin for 
print('Los MSE para scikit-learn son: ')
print(MSE_p1)
#print(X_entrenamiento['skoda'].loc[[908,2908]])
#print(X_entrenamiento['skoda'].loc[[908,2908]])
#print(len(X_entrenamiento['skoda'].index))
#print(list(y_entrenamiento['skoda'].index))

audi :  {'intercepto': 338245972781.5106, 'year': 0.3413111519483319, 'mileage': -0.1596267075328024, 'tax': -0.17092979608895717, 'mpg': -0.311409082100568, 'engineSize': 0.2330923533511535, ' R8': 6023766949.437706, ' A8': 6023766944.992901, ' RS5': 6023766945.969752, ' S4': 6023766944.926721, ' Q3': 6023766944.514072, ' S8': 6023766944.907708, ' S5': 6023766944.388247, ' A7': 6023766944.678456, ' A2': -1666062232.5802178, ' RS4': 6023766946.028512, ' Q5': 6023766944.844244, ' SQ5': 6023766945.01711, ' A5': 6023766944.51039, ' RS7': 6023766945.917722, ' A3': 6023766944.363455, ' A4': 6023766944.394299, ' Q8': 6023766946.375343, ' TT': 6023766944.553466, ' RS6': 6023766946.618801, ' A1': 6023766944.2575655, ' Q2': 6023766944.382711, ' RS3': 6023766945.100872, ' A6': 6023766944.567674, ' S3': 6023766944.768196, ' Q7': 6023766945.56913, ' SQ7': 6023766945.722907, 'Automatic': -210292288712.5656, 'Manual': -210292288712.70154, 'Semi-Auto': -210292288712.558, 'Petrol': -133977451013.48685

<b> Implementación del descenso del gradiente estocástico </b>
<p><b> Al igual que en la parte a se hizo una regresión para cada marca </b></p>

<p> En esta primera celda se introdujo una columna de unos como la primera columna de los datos, para que el "w" calculado a partir de esta columna sea el intercepto del modelo </p>

In [8]:
#Insertar unos como la primera columna de las matrices X_entrenamiento y X_test
for marca in marcas:
    X_entrenamiento[marca].insert(loc = 0, column = 'intercepto', value = np.ones(len(DatosEntrenamiento[marca][feature_y])))
    X_test[marca].insert(loc = 0, column = 'intercepto', value = np.ones(len(DatosTest[marca][feature_y])))
#fin for 

<p><b> La siguiente celda define la función que determinará un "w" a partir de unos datos de entrenamiento (X) y una columna de target (y). A esta función se le puede definir la tasa de 
aprendizaje, el número de minibatches y el número máximos de iteraciones. Si estos no se definen, por defecto la tasa de aprendizaje tendrá un valor de 0.1, el número de minibatches será de 100 y las iteraciones máximas serán de 30. En cada iteración la función ordena aleatoriamente cada fila de datos y los divide en N_batches minibatches. Luego calcula el gradiente para el primer minibatch y determina un nuevo w a partir de ese gradiente, luego va al siguiente minibatch, determina el gradiente y vuelve a determinar un nuevo w, así sucesivamente hasta que pasa por todos los minibatches. Una vez pasa por todos los minibatches vuelve a repetir el proceso (volviendo a reordenar los datos aleatoriamente) el número de veces que permitan las máximas iteraciones "max_iter". </b></p>

In [9]:
def hallar_w_descenso_gradiente(X,y,w0, max_iter=30, N_batches=100, alpha1=0.1):
    """
    Aquí se define el tamaño del minibatch en la variable de "tam_batch"
    Esta función determina los w usando el método de descendo de gradiente estocástico. Lo primero que hace es reordenar las filas aleatoriamente. Para ello se creó una lista de indices, la 
    cual contiene números enteros ordenados al azar. Luego cada minibatch va a tomar los datos correspondientes a los índices asignados. Por ejemplo si el tamaño del minibatch es de 5 
    y los primeros 10 elementos de "indices" son: [35, 475, 11, 232, 99, 10, 0, 87, 44, 29], entonces el primer minibatch estará compúesto de los datos de las filas 35, 475, 11, 232 y 99, 
    mientras que el segundo estará compuesto de las filas 10, 0, 87, 44, 29, y así sucesivamente para los siguiente minibatches. 
    @input: X <pd.DataFrame>: Datos de entrenamiento (sin el target)
    @input: y <pd.DataFrame>: Columna de los target de los datos de entrenamiento 
    @input: (opcional) max_iter <int>: Número máximo de iteraciones
    @input: (opcional) N_batches <int>: Número de minibatches en los que se dividirán los datos
    @input: (opcional) alpha1 <float>: Tasa de aprendizaje
    @output: w <np.ndarray>: Los pesos calculados a partir de la regresión lineal con descenso de gradiente estocástico
    """
    tam_batch = int(len(y)/N_batches)
    w = w0
    for n_iter in range(max_iter):
        indices = sorted(X.index, key = lambda x: np.random.rand())
        for i in range(N_batches):
            if i < N_batches -1:
                indices_loc = indices[i*tam_batch:(i+1)*tam_batch]
            else:
                indices_loc = indices[i*tam_batch:]
            #fin if 
            X_batch = X.loc[indices_loc]
            y_batch = y.loc[indices_loc]
            gradiente = ((X_batch.T).dot(X_batch)).dot(w) - (X_batch.T).dot(y_batch)
            w = w - alpha1*gradiente
        #fin for
    #fin for
    return w
#fin función

<p> En la siguiente celda se aplica la función "hallar_w_descenso_gradiente" a cada una de las marcas. El criterio de inicialización de los w se explica en un comentario dentro de la celda. Los resultados de los w se guardan en un diccionario llamado w_p2, los y_calculados con los datos de test se guardan en un diccionario llamado y_gorro_test_p2, y finalmente los MSE se guardan en el diccionario MSE_p2 </p>

In [10]:
"""
NOTA: Con una CPU de 16 hilos esto se demora 49s en correr. Si le bajo al número de batches o le subo a la taza de aprendizaje tan solo un poquito, todo se va al @#@⅛! para todas las marcas
SI le bajo a la taza de aprendizaje y le subo al número de batches así como al número máximo de iteraciones esto se demora siglos. Aún así simpre hay una marca cuyo MSE da horrible
***************************
Criterio de Inicialización:
En cuanto al intercepto la mejor inicialización es 0, ya que este puede ser negativo o positivo, y no se sabe mas
Entre mayor sea el año, es decir mas nuevo el carro se epera que sea mas caro, se le inicializará con +0.5
Entre mas usado sea el carro, es decir mayor sea su "mileage" se espera que sea mas barato, se inicializará con -0.5
Entre mas se cobren impuestos se espera que valga menos: -0.5
Entre mas eficiente sea un carro, es decir, entre mas millas por galón recorra, se espera que valga mas: +0.5
Entre mas grande sea el motor, se espera que sea mas caro: +0.5
En cuanto a los modelos de los carros es muy complicado conocer a priori el signo de la pendiente. Todos estos se inicializarán con 0
Se espera que si un carro es automático, este sea mas caro, respecto a los otros casos, por lo que se inicializará su pendiente con 0.5
Si es semiautomático se espera que estén en intermedio de precios por lo que se inicializará con 0
Si es manual se espera que sea mas batato que el automático o semi, por que se inicializará con -0.5 su pendiente
Los demás se inicializarán desde 0

"""
MSE_p2 = dict()
w_p2 = dict()
y_gorro_test_p2 = dict()
resultados_p2 = dict()
for marca in marcas:
    n_caracteristicas = len(X_entrenamiento[marca].keys())
    w0 = np.array([0, 0.5, -0.5, -0.5, 0.5, 0.5] + [0]*(n_caracteristicas - 13) + [0, 0.5, -0.5] + [0]*4)
    resultados_p2[marca] = dict(hallar_w_descenso_gradiente(X_entrenamiento[marca], y_entrenamiento[marca], w0, max_iter=150, N_batches=7, alpha1=0.0004))
    w_p2[marca] = list(resultados_p2[marca].values())
    y_gorro_test_p2[marca] = X_test[marca].dot(w_p2[marca])
    MSE_p2[marca] = 1/(len(y_test[marca]))*((y_test[marca] - y_gorro_test_p2[marca]).dot(y_test[marca] - y_gorro_test_p2[marca]))
#fin for 


<p><b> Finalmente en las siguientes dos celdas se reportan los "w" obtenidos tanto en el punto con scikit-learn como con el punto con la propia implementación de descnso de gradiente estocástico. También se reportan los MSE para cada marca y para cada método </b></p>

In [11]:
MSE = dict()
for marca in marcas:
    print('------- Resultados de la marca: ', marca + ' ' +'-'*200)
    print('Scikit-learn: ', resultados_sklearn['skoda'])
    print('D.Gradiente Est.: ', resultados_p2['skoda'])
    print('-'*250)
    MSE[marca] = [MSE_p1[marca], MSE_p2[marca]]
#fin for
MSE = pd.DataFrame(MSE, index=['Scikit-learn', 'D.Gradiente Est.']).T

------- Resultados de la marca:  audi --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Scikit-learn:  {'intercepto': 1.4123168998950328, 'year': 0.273981725950823, 'mileage': -0.1812629715038298, 'tax': -0.029034919914858064, 'mpg': -0.34704462724750057, 'engineSize': 0.19309839218987915, ' Scala': -0.0742436925663115, ' Superb': 0.2757494896150686, ' Rapid': -0.3084895033970586, ' Octavia': -0.049618683854954285, ' Yeti': -0.22146701077514136, ' Roomster': -0.42662087386527847, ' Fabia': -0.2941283047345885, ' Kodiaq': 0.8436909258921683, ' Yeti Outdoor': -0.13791280495848643, ' Kamiq': 0.2760335274673769, ' Karoq': 0.5300534580485308, ' Citigo': -0.4130465268713183, 'Automatic': 0.167870278717535, 'Manual': -0.09231716169886114, 'Semi-Auto': 0.14428461095233036, 'Other': -1.3773221030584952, 'Petrol': -1.4991583388607055, 'Hybrid': 4.

In [12]:
print(MSE)

          Scikit-learn  D.Gradiente Est.
audi      2.771016e+16      1.214588e-01
bmw       1.219173e-01      1.265187e-01
ford      1.938699e-01     6.709349e+139
hyundi    1.199799e-01      1.223964e-01
merc      1.857752e-01      1.938924e-01
skoda     7.572449e-02      8.840013e-02
toyota    7.290788e-02      9.206250e-02
vauxhall  1.453244e-01      1.511044e-01
vw        1.076397e-01      1.087639e-01
